In [ ]:
!pip install yfinance -q
import yfinance as yf
import numpy as np
import pandas as pd
from pprint import pprint

!pip install pyomo -q
!apt-get install -y -qq coinor-cbc -q
!pip install pyomo[solvers] -q
from pyomo.environ import *
import pyomo.environ as pyo
from pyomo.environ import SolverFactory
from pyomo.opt import SolverFactory

!apt-get install -y coinor-cbc




Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
coinor-cbc is already the newest version (2.10.7+ds1-1).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.


In [ ]:
tickers = ['AAPL', 'MSFT', 'GOOGL'] #'SAP','RMS.PA', 'BABA', 'BND','GLD','BTC']
model = pyo.ConcreteModel(name='Financial_Portfolio')
model.assets = Set(initialize=tickers)

data1 = yf.download(tickers, start='2019-12-31', end='2025-01-02')
data=pd.DataFrame(data1)
closing_prices=data['Close']

closing_prices.index=pd.to_datetime(closing_prices.index)
return_pct_daily=closing_prices.pct_change().dropna()
returns=(return_pct_daily.mean())*1000

cost1=(yf.download(tickers, start='2025-01-02', end='2025-01-03'))
data=pd.DataFrame(cost1)
closing_prices=np.squeeze(data['Close'])

cov_matrix = return_pct_daily.cov().values

Lminall=[0.05, 0.2, 0.3]
Lmaxall=[0.8, 0.7, 0.9]


B= 30000 #5 capital budget
Riskmax= 990  #6 riskmax
K=0 #min num of asset must be equal to the number of a
alpha= 0.5 #propensione al rischio

parameters= {#'returns': {tickers: value for tickers, value in zip(tickers, returns)},
             'returns': {tickers[i]: returns.iloc[i] for i in range(len(tickers))},
             'costs': {tickers[i]: closing_prices.iloc[i] for i in range(len(tickers))},
              'L_min_allocations': {tickers: value for tickers, value in zip(tickers, Lminall)},
             'U_max_allocations': {tickers: value for tickers, value in zip(tickers, Lmaxall)},
             "Budget": B,
             "Risk_Max": Riskmax,
             "Min_Assets": K,
             "Alpha": alpha,
             }
print('\n\n')
for key, value in parameters['returns']:
    print(f"Key: {key}, Value: {value}, Type: {type(value)}")
print(type(parameters['returns']))
# Add parameters to the model
model.returns = pyo.Param(model.assets, initialize=parameters['returns'], within=Any)
model.costs = pyo.Param(model.assets, initialize=parameters['costs'], within=Any)
#scalar parameters (non associati ad un set)
model.Budget = pyo.Param(initialize=parameters['Budget'])

print('\n\n')
pprint(parameters)





#model.y = pyo.Var(model.assets, domain=pyo.Binary)

model.x = Var(model.assets, domain=pyo.NonNegativeReals, bounds=(0, B), initialize=0)
#model.x = pyo.Var(model.assets, domain=pyo.NonNegativeReals, bounds=(0, 1)) # Bounds added: 0 <= x <= 1

'''CONSTRAINTS'''
#1
model.Budget_constraint= pyo.Constraint(expr=sum
                                  (model.costs[i]*model.x[i]
                                    for i in model.assets) <= model.Budget)

def portfolio_objective(model):
    expected_return = sum(model.returns[i] * model.x[i]
                          for i in model.assets)
    return expected_return

model.Objective = pyo.Objective(rule=portfolio_objective, sense=pyo.maximize)

model.pprint()



results=pyo.SolverFactory('cbc').solve(model, tee=True)
print(results.solver.status)   # OK, Warning, Error, etc.
print(results.solver.termination_condition)  # Optimal, Infeasible, Unbounded, etc.

# Estrarre i valori delle variabili ottimizzate
print("\nOptimal solution:")
for asset in model.assets:
    print(f"{asset}: {model.x[asset].value}")

# Estrarre il valore della funzione obiettivo
#print("\nOptimal objective value:", model.Objective.expr())

[*********************100%***********************]  3 of 3 completed
[*********************100%***********************]  3 of 3 completed

ValueError: too many values to unpack (expected 2)

In [ ]:
print(results.solver.status)   # OK, Warning, Error, etc.
print(results.solver.termination_condition)  # Optimal, Infeasible, Unbounded, etc.

# Estrarre i valori delle variabili ottimizzate
print("\nOptimal solution:")
for asset in model.assets:
    print(f"{asset}: {model.x[asset].value}")

# Estrarre il valore della funzione obiettivo
print("\nOptimal objective value:", model.Objective.expr())


ok
optimal

Optimal solution:
AAPL: 0.0
MSFT: 158.36985
GOOGL: 0.0

Optimal objective value: 164.51818624397413


In [ ]:
!pip install pyomo
!apt-get install -y coinor-cbc

In [ ]:
import pyomo.environ as pyo
import yfinance as yf
import pandas as pd
import numpy as np

# Definizioni iniziali
tickers = ['AAPL', 'MSFT', 'GOOGL']
B = 30000  # Budget disponibile
# Scarica i dati
data1 = yf.download(tickers, start='2019-12-31', end='2025-01-02')
closing_prices = data1['Close']
closing_prices.index = pd.to_datetime(closing_prices.index)
return_pct_daily = closing_prices.pct_change().dropna()
returns_series = return_pct_daily.mean()*100000
returns = returns_series.to_dict() #return is a dict!!


cost1 = yf.download(tickers, start='2025-01-02', end='2025-01-03')
costs_df = cost1['Close']
print('\n\n')
print((costs)) #cost è un data frame
#costs.iloc[0].to_dict()
costs = costs_df.squeeze().to_dict() #cost è un dict

max_return = max(returns.values())
max_cost = max(costs.values())


# Dichiara il modello
model = pyo.ConcreteModel(name='Financial_Portfolio')

# Dichiara gli asset
model.assets = pyo.Set(initialize=tickers)
model.x = pyo.Var(model.assets, domain=pyo.NonNegativeReals)
model.returns = pyo.Param(model.assets, initialize=returns)
model.costs = pyo.Param(model.assets, initialize=costs)
model.x.display()
model.minconstraint= pyo.Constraint(expr= sum(model.x[i] for i in model.assets)>= 5)
# Vincolo di budget: la somma dei costi di tutti gli asset acquistati deve essere <= al budget
def budget_constraint(model):
    return sum(model.costs[i] * model.x[i] for i in model.assets) <= B
model.budget_constraint = pyo.Constraint(rule=budget_constraint)

# Funzione obiettivo: massimizzare il ritorno del portafoglio
def objective_function(model):
    return sum(model.returns[i] * model.x[i] for i in model.assets)
model.obj = pyo.Objective(rule=objective_function, sense=pyo.maximize)

# Risolvi il problema
solver = pyo.SolverFactory('cbc')
results=solver.solve(model, tee=True)

# Stampa i risultati
print(f"Soluzione ottimale:")
for asset in model.assets:
    print(f"{asset}: {model.x[asset].value:.2f} units")

print(f"Valore dell'obiettivo (ritorno complessivo): {model.obj():.4f}")

model.pprint()

[*********************100%***********************]  3 of 3 completed
[*********************100%***********************]  3 of 3 completed




{'AAPL': 243.85000610351562, 'GOOGL': 189.42999267578125, 'MSFT': 418.5799865722656}
x : Size=3, Index=assets
    Key   : Lower : Value : Upper : Fixed : Stale : Domain
     AAPL :     0 :  None :  None : False :  True : NonNegativeReals
    GOOGL :     0 :  None :  None : False :  True : NonNegativeReals
     MSFT :     0 :  None :  None : False :  True : NonNegativeReals
Welcome to the CBC MILP Solver 
Version: 2.10.7 
Build Date: Feb 14 2022 

command line - /usr/bin/cbc -printingOptions all -import /tmp/tmp7m99wjtt.pyomo.lp -stat=1 -solve -solu /tmp/tmp7m99wjtt.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
 CoinLpIO::readLp(): Maximization problem reformulated as minimization
Coin0009I Switching back to maximization to get correct duals etc
Presolve 2 (0) rows, 3 (0) columns and 6 (0) elements
Statistics for presolved model


Problem has 2 rows, 3 columns (3 with objective) and 6 elements
Column breakdown:
3 of type 0.0->inf, 0 of type 0.0

In [ ]:
print(costs_dict)  # Verifica se ci sono valori None o NaN


{'AAPL': 243.85000610351562, 'GOOGL': 189.42999267578125, 'MSFT': 418.5799865722656}


In [ ]:
print([v for v in model.x])



['AAPL', 'MSFT', 'GOOGL']


In [ ]:
print(model.obj.expr)


119.93076508809854*x[AAPL] + 100.17545589557479*x[MSFT] + 103.88226436027698*x[GOOGL]


In [ ]:
print(model.budget_constraint)


budget_constraint
